<a href="https://colab.research.google.com/github/HenryLiangzy/COMP9417_Project/blob/master/Henry_s_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt


In [0]:
np.random.seed(1)
TRAINING_FILE = "training.csv"
TEST_FILE = "test.csv"

In [0]:
df = pd.read_csv(TRAINING_FILE)
print(df.shape[0], df.shape[1])

9500 3


In [0]:
train_set, test_set = train_test_split(df, test_size=0.2)
print(train_set)

      article_number  ...          topic
7786            7787  ...  FOREX MARKETS
8777            8778  ...     IRRELEVANT
6370            6371  ...         SPORTS
6049            6050  ...     IRRELEVANT
4396            4397  ...     IRRELEVANT
...              ...  ...            ...
2895            2896  ...  MONEY MARKETS
7813            7814  ...     IRRELEVANT
905              906  ...         SPORTS
5192            5193  ...     IRRELEVANT
235              236  ...         SPORTS

[7600 rows x 3 columns]
